In [1]:
import pandas as pd
from DataProcessing import main_data_processing, distance_formula
import plotly.graph_objects as go
outside_assignment_df = pd.read_table('/Volumes/Transcend/fawzi_pycharm_project/fto_assignment_files/Full_length/Run/assigned_peaks_res.out', sep=' +')
outside_peaks_df = pd.read_table('/Volumes/Transcend/fawzi_pycharm_project/fto_assignment_files/Full_length/CS17_FTO', sep=' +', index_col=False)
peak_list, residue_list = main_data_processing()

outside_peaks_df['N_'] = [float(x)+.23 for x in outside_peaks_df['N']]
outside_peaks_df['H_'] = [float(x)+.08 for x in outside_peaks_df['H']]
outside_peaks_df['CO-1'] = [float(x) if x != '-' else '-' for x in outside_peaks_df['CO-1']]

/Users/sohamkale/miniconda3/envs/Fawzi_pycharm_project/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.

/Users/sohamkale/miniconda3/envs/Fawzi_pycharm_project/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [2]:
old_match_list = [[] for x in range(len(outside_peaks_df))]
H_tolerance = .05
N_tolerance = .15
C_tolerance = .1
total = 0
no_match = 0
nh_match = 0
co_discrim = 0
no_co_discrim = 0
ambiguous = 0
for peak in peak_list:
    h_shift = peak.get_data('TROSYHShift')
    n_shift = peak.get_data('TROSYNShift')
    if h_shift is not None:
        total += 1
        ca_shift = peak.get_data('CAShift')
        cb_shift = peak.get_data('CBShift')
        ca_prime_shift = peak.get_data('CAPrimeShift')
        cb_prime_shift = peak.get_data('CBPrimeShift')
        co_prime_shift = peak.get_data('COPrimeShift')
        df = outside_peaks_df[(abs(outside_peaks_df['N_'] - n_shift) < N_tolerance) & (abs(outside_peaks_df['H_'] - h_shift) < H_tolerance)]
        # if there are no matches
        if len(df) == 0:
            # print('no matches for')
            no_match += 1
        # if there is exactly one match
        elif len(df) == 1:
            nh_match += 1
            old_match_list[df.index[0]].append(peak)
        # if it is ambiguous
        elif len(df) > 1:
            # see if co discrimination works
            # checks to see if all the new peak have CO shifts
            if co_prime_shift is not None and len(df[df['CO-1'] == '-']) == 0:
                df = df[(abs(df['CO-1'] - co_prime_shift) < 1)]
                # successful co discrimination case
                if len(df) == 1:
                    co_discrim += 1
                    old_match_list[df.index[0]].append(peak)
                # unsuccessful
                else:
                    ambiguous += 1
                    print('h:', h_shift, 'n:', n_shift, 'ca:', ca_shift, 'ca-1', ca_prime_shift,
                                          'cb:', cb_shift, 'cb-1:', cb_prime_shift, 'co-1:', co_prime_shift)
                    print(df, '\n\n')
            else:
                print('h:', h_shift, 'n:', n_shift, 'ca:', ca_shift, 'ca-1', ca_prime_shift,
                      'cb:', cb_shift, 'cb-1:', cb_prime_shift, 'co-1:', co_prime_shift)
                print(df, '\n\n')
                ambiguous += 1
print('total:', total, 'NH Match:', nh_match,
      'no match:', no_match, 'co discrim:', co_discrim, 'ambiguous:', ambiguous)

outside_peaks_df['old match'] = old_match_list

h: 7.394 n: 121.492 ca: 58.627 ca-1 59.915 cb: 54.416 cb-1: 55.296 co-1: 180.526
     peaknumber     CA   CA-1     CB   CB-1   CO-1      H      N      N_  \
19           21  58.56  59.78  30.52  30.16  180.5  7.320  121.3  121.53   
404        2004  54.16  52.99      -      -  175.4  7.337  121.3  121.53   
406        2033      -      -      -  18.62      -  7.341  121.3  121.53   

        H_  
19   7.400  
404  7.417  
406  7.421   


h: 8.854 n: 128.462 ca: 57.857 ca-1 59.3 cb: None cb-1: None co-1: 176.643
     peaknumber     CA   CA-1     CB   CB-1   CO-1      H      N      N_  \
50           59  57.84      -  27.49  32.08  176.5  8.753  128.3  128.53   
309        1002  59.14  48.72      -      -      -  8.732  128.2  128.43   

        H_  
50   8.833  
309  8.812   


h: 8.857 n: 123.893 ca: 59.004 ca-1 54.62 cb: None cb-1: None co-1: 176.159
Empty DataFrame
Columns: [peaknumber, CA, CA-1, CB, CB-1, CO-1, H, N, N_, H_]
Index: [] 


h: 8.782 n: 117.049 ca: 51.209 ca-1 53.574 cb:

In [114]:
a, b, c  = 0, 0, 0
for index in range(len(outside_peaks_df)):
    if len(outside_peaks_df['old match'][index]) == 1:
        peak = outside_peaks_df['old match'][index][0]
        if peak.get_data('CAShift') is not None and outside_peaks_df['CA'][index] == '-':
            # print(outside_peaks_df.iloc[index, :])
            # print(peak.peak_properties)
            # print('\n\n')
            a += 1
        if peak.get_data('CAPrimeShift') is not None and outside_peaks_df['CA-1'][index] == '-':
            # print(outside_peaks_df.iloc[index, :])
            # print(peak.peak_properties)
            # print('\n\n')
            b += 1
        if len(peak.get_data('NearbyHShift')) != 0:
            print(outside_peaks_df.iloc[index, :])
            print(peak.get_data('NearbyHShift'))
            print('\n\n')
            c += 1

(a,b,c)


peaknumber                                                    1
CA                                                        43.33
CA-1                                                      51.69
CB                                                            -
CB-1                                                      16.52
CO-1                                                      177.7
H                                                         2.881
N                                                           108
N_                                                       108.23
H_                                                        2.961
old match     [<DataProcessing.Peaks object at 0x7fb7f97f1d50>]
Name: 0, dtype: object
[8.496, 7.63, 7.875, 10.944, 9.763]



peaknumber                                                    3
CA                                                            -
CA-1                                                          -
CB                                        

(6, 18, 227)

In [12]:
d = int()
for index in range(len(outside_peaks_df)):
    if len(outside_peaks_df['old match'][index]) > 1:
        d += 1
        print(outside_peaks_df.loc[index, : ])
        for peak in outside_peaks_df.loc[index, 'old match']:
            print('H:', peak.get_data('TROSYHShift'), 'N:', peak.get_data('TROSYNShift'),
                  'CA:', peak.get_data('CAShift'), 'CA-1:', peak.get_data('CAPrimeShift'),
                  'CO-1:', peak.get_data('COPrimeShift'), 'NOESY', peak.get_data('NOESYHShift'))
        print('\n\n')



peaknumber                                                   34
CA                                                        49.07
CA-1                                                      45.58
CB                                                        20.14
CB-1                                                          -
CO-1                                                      173.7
H                                                         6.915
N                                                           118
N_                                                       118.23
H_                                                        6.995
old match     [<DataProcessing.Peaks object at 0x7fd3abc0415...
Name: 32, dtype: object
H: 6.99 N: 118.313 CA: 49.098 CA-1: 57.527 CO-1: 173.85 NOESY [7.007, 7.0, 7.007, 7.004]
H: 6.956 N: 118.09 CA: None CA-1: None CO-1: None NOESY []



peaknumber                                                   93
CA                                                       